In [1]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt
from tqdm import tqdm
from _funcs import *

In [15]:
class DiffusiveEvolutionPnt:
    """
    This class is used to compute the diffusive evoltuion of the N resolved density operator
    We use vectorised density operators and the projective evolution of the jump operator

    System evolves according to 
    d rho_n / dt = L rho_n + H (d rho_n / d N) + (K_d /2) (d^2 rho_n / d N^2)

    where L is the Liouvillian, H is measurement superoperator, K_d = 1 is the dynamical activity (for this simple case)

    NOTE: For time being, this class is only for a single collapse operator
    """

    def __init__(self, H: Qobj, c_ops: list, t: np.ndarray, N: np.ndarray):
        """
        Parameters
        ----------
        H : qutip.Qobj
            The system Hamiltonian
        c_ops : list of qutip.Qobj
            The list of collapse operators
        t : list of float
            The list of times
        N : list of float
            The list of N values
        """
        self.H = H
        self.c_ops = c_ops
        self.t = t
        self.N = N
        self.N_len = len(N)
        self.dt = t[1] - t[0]   # assuming uniform grid
        self.dN = N[1] - N[0]   # assuming unifrom grid
        self.dim = H.shape[0]**2    # dimension of the Liouvillian space

        assert len(c_ops) == 1, f"Only one collapse operator is supported for time being!"

    def measurement_superoperators(self):
        """
        Return the Kraus operators for the qubit diffusion operator
        """
        L = (1+ self.dt * liouvillian(self.H, self.c_ops) - self.dt/self.dN**2).full()
        H_op1 = ((self.dt/(2*self.dN))*(
            spre(self.c_ops[0])   + self.dt * spost(self.c_ops[0].dag())  + 1/self.dN
                                        )
                                        ).full()
        H_op2 = ((self.dt/(2*self.dN))*(
            -spre(self.c_ops[0]) - self.dt * spost(self.c_ops[0].dag())  + 1/self.dN
                                        )
                                        ).full()

        return [L, H_op1, H_op2]
    
    def evolution_matrix(self):

        """
        Implements an absorbing boundary condition on the n-resolved density matrix 

        M[0] reserved for normal evolution
        M[1] reserved for diffusion operator 

        Returns
        -------
        M_update : np.array
            The evolution matrix
        """

        M = self.measurement_superoperator()
        nu_k = [0, 1, -1]

        # Compute M_update superoperats
        M_update_ops = [np.kron(np.diag(np.ones(self.N_len - np.abs(nu_k[i])),  k=nu_k[i]), M[i]) for i in range(len(nu_k))]
        M_update = sum(M_update_ops)
    
        return M_update
    
    def solve(self, rho0, ix=0):
        """
        Solve the projective evolution of the density matrix

        Parameters
        ----------
        rho0 : Qobj
            The initial density matrix
        ix : int        
            The index of the initial state in the n-resolved basis

        Returns
        -------
        Pnt : np.array
            Solution to the n-resolved density matrix
        """

        # Compute the evolution matrix
        M_update = self.evolution_matrix(nu_k)

        # Convert the initial state to a vector 
        if rho0.type == 'oper':
            print("Converting initial state to vector form")
            rho0 = operator_to_vector(rho0).full()
        elif rho0.type == 'ket':
            print("Converting initial state to vector form")
            rho0 = operator_to_vector(ket2dm(rho0)).full()
        else:
            print("Initial state is already in vector form")
            rho0 = rho0.full()

        # Initialise the density matrix vector
        rho_n_vec = np.zeros((self.dim*self.N_len, len(self.t)), dtype=complex)
        rho_n_vec[self.dim*ix:self.dim*(ix+1), 0] = rho0.flatten()

        # Get Ivec
        Ivec = np.eye(int(np.sqrt(self.dim))).reshape(self.dim,)

        # Initialise Pn
        Pn_vec = np.zeros((self.N_len, len(self.t)))
        Pn_vec[:, 0] = [np.real(np.dot(Ivec, rho_n_vec[self.dim*n:self.dim*(n+1), 0])) for n in range(self.N_len)]

        # evolve rho
        for i in tqdm(range(1, len(self.t)), desc="Evolution Superoperator"):
            rho_n_vec[:, i] = M_update @ rho_n_vec[:, i-1] 

            # calculate Pn
            for j in range(self.N_len):
                Pn_vec[j, i] = np.real(np.dot(Ivec, rho_n_vec[self.dim*j:self.dim*(j+1), i]))

        return Pn_vec
    

In [13]:
# Parameters
Gamma = 0.2
Omega = 1
dt = 0.001
t = np.arange(0, 2, dt)

# Define system operators
sp = sigmap()
sm = sigmam()
sx = sigmax()
sz = sigmaz()
H = Omega*sx

# define dissipator and Hamiltonian
c_ops = [np.sqrt(Gamma) * sz]    

# number of chargers to truncate at 
Nm = 5
dN = 0.1
# N_cutoff = 10
N = np.arange(-Nm, Nm, dN)

# Initial index
i0 = np.argmin(np.abs(N))


In [16]:
diff = DiffusiveEvolutionPnt(H, c_ops, t, N)

AssertionError: Only one collapse operator is supported for time being!

In [17]:
len(c_ops)

1